In [1]:
##4QAM, 16QAM, 64QAM, BPSK, 8PSK.

In [5]:
from keras import backend as K
from keras.layers import Layer
from keras.layers import Activation, Dense
from keras import optimizers
import numpy as np
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt

In [63]:
#Definitions to automatically generate symbols of QAM and M-Ary PSK
def gray_code(n):
    if n < 1:
        g = []
    else:
        g = ['0', '1']
        n -= 1
        while n > 0:
            k = len(g)
            for i in range(k-1, -1, -1):
                char = '1' + g[i]
                g.append(char)
            for i in range(k-1, -1, -1):
                g[i] = '0' + g[i]
            n -= 1
    return g

class QAM:
    def __init__(self, NUM_SYMBOLS = 4, SAMP_RATE = 1):
        self.NUM_SYMBOLS = NUM_SYMBOLS
        self.SAMP_RATE = SAMP_RATE
        dim_iq_axis = int(np.sqrt(NUM_SYMBOLS))
        self.coords = []
        self.symbols = gray_code(dim_iq_axis)
        MAX_ENERGY = np.sqrt(2) #For QAM with coordinates at (x=1,y=1), normalized energy: divide by sqrt(2)
        #4 Syms: Step size is 2
        #16 Syms: Fit 4 syms one line from -1 to 1. Step size is 1/2
        #We fit 4 since it's 16 Symbols. On one line, it's sqrt(NUM_SYMBOLS) = 4
        #Symbols = Spacing * Width (Width = 2). So spacing = symbols / width
        print(np.arange(-1, 1.1, 4/(dim_iq_axis)))
        for i in np.arange(-1, 1.1, 4/(dim_iq_axis)):
            for k in np.arange(-1, 1.001, 2/(dim_iq_axis)):
                self.coords.append(np.complex64(i + k*1j)/(MAX_ENERGY))
        
        #self.bits_to_coords = dict(zip(self.symbols, self.coords))
        print('coord:', self.coords, "\n")
        #print(self.bits_to_coords)
        #Above generates a grid structure for QAM
        
class PSK:
    def __init__(self, NUM_SYMBOLS = 8, SAMP_RATE = 1):
        self.NUM_SYMBOLS = NUM_SYMBOLS
        self.SAMP_RATE = SAMP_RATE
        self.grid = np.zeros(NUM_SYMBOLS)
        #self.symbols = gray_code(NUM_SYMBOLS)
        self.symbols = gray_code(np.log2(NUM_SYMBOLS))
        cnt = 0
        for i in np.arange(0, NUM_SYMBOLS/(2*np.pi) , 2*np.pi):
            #i is the phase
            self.grid[cnt] =  1*np.exp(1j*i)
            cnt += 1
        #Above generates an array with the approximate locations of the symbols.
        print(self.symbols)
        
qam4 = QAM(NUM_SYMBOLS=4)
qam16 = QAM(NUM_SYMBOLS=16)
qam64 = QAM(NUM_SYMBOLS=64)
BPSK = PSK(NUM_SYMBOLS = 2)
PSK8 = PSK(NUM_SYMBOLS = 8)

[-1.  1.]
coord: [(-0.7071067811865475-0.7071067811865475j), (-0.7071067811865475+0j), (-0.7071067811865475+0.7071067811865475j), (0.7071067811865475-0.7071067811865475j), (0.7071067811865475+0j), (0.7071067811865475+0.7071067811865475j)] 

[-1.  0.  1.]
coord: [(-0.7071067811865475-0.7071067811865475j), (-0.7071067811865475-0.35355339059327373j), (-0.7071067811865475+0j), (-0.7071067811865475+0.35355339059327373j), (-0.7071067811865475+0.7071067811865475j), -0.7071067811865475j, -0.35355339059327373j, 0j, 0.35355339059327373j, 0.7071067811865475j, (0.7071067811865475-0.7071067811865475j), (0.7071067811865475-0.35355339059327373j), (0.7071067811865475+0j), (0.7071067811865475+0.35355339059327373j), (0.7071067811865475+0.7071067811865475j)] 

[-1.  -0.5  0.   0.5  1. ]
coord: [(-0.7071067811865475-0.7071067811865475j), (-0.7071067811865475-0.5303300858899106j), (-0.7071067811865475-0.35355339059327373j), (-0.7071067811865475-0.17677669529663687j), (-0.7071067811865475+0j), (-0.707106781

d:\applications\python3\lib\site-packages\ipykernel_launcher.py:50: ComplexWarning: Casting complex values to real discards the imaginary part


In [6]:
def generate_model(bps = 4):
    model = Sequential()
    model.add(Dense(bps, input_dim=bps))
    model.add(Activation('relu'))
    model.add(Dense(bps, input_dim=bps))
    return model

def soft_demodulate(num_bits_symbol, symbol_mapping):
    #Code taken from CommPy: https://github.com/veeresht/CommPy/blob/master/commpy/modulation.py
    demod_bits = np.zeros(len(input_symbols) * num_bits_symbol)
    for i in arange(len(input_symbols)):
        current_symbol = input_symbols[i]
        for bit_index in arange(num_bits_symbol):
            llr_num = 0
            llr_den = 0
            for const_index in symbol_mapping:
                if (const_index >> bit_index) & 1:
                    llr_num = llr_num + exp(
                        (-abs(current_symbol - self.constellation[const_index]) ** 2) / noise_var)
                else:
                    llr_den = llr_den + exp(
                        (-abs(current_symbol - self.constellation[const_index]) ** 2) / noise_var)
            demod_bits[i * self.num_bits_symbol + self.num_bits_symbol - 1 - bit_index] = log(llr_num / llr_den)
    return demod_bits

NameError: name 'model' is not defined

In [ ]:
Adam=optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='cross-entropy', optimizer='Adam', metrics=['accuracy'])
training_data = np.array(X_train)
test_data = np.array(X_test)

#Run model. Make sure to regen this model to prevent overfitting
history=model.fit(training_data, y_train, validation_data=(test_data, y_test), epochs=130)
history_list.append(history)